In [1]:
import numpy as np
import pandas as pd
import random
import time
import tensorflow as tf
from six.moves import cPickle as pickle


In [2]:
train_is = np.genfromtxt ('train_numeric.csv', delimiter=",", skip_header=2001, usecols = (0,), dtype=int)


In [34]:
cols = pd.read_csv('train_numeric.csv', index_col=0, nrows=1, low_memory=False)
cols_d = dict()
for i, c in enumerate(cols.columns):
    cols_d[c] = i


In [3]:
train_len = len(train_is)
cat_features = 94

In [4]:
num = pd.read_csv('train_numeric.csv', index_col=0, skiprows=1, header=None, nrows=2000, dtype=np.float32)
num.fillna(0., inplace=True)
cat = pd.read_csv('train_categorical_c.csv', index_col=0, skiprows=1, header=None, nrows=2000, dtype=np.float32)
cat = cat.div(cat_features,axis='columns')
(num.shape, cat.shape)

((2000, 969), (2000, 2140))

In [41]:
# num.groupby(cols_d['L1_S24_F1846'])[cols_d['Response']].mean().reset_index()

In [5]:
# data = pd.concat([cat, num], axis=1)
data = num
X_valid = data.values[:,:-1]
y_valid = data.values[:,-1]
(X_valid.shape, y_valid.shape)

((2000, 968), (2000,))

In [6]:


data_num_chunks = pd.read_csv('train_numeric.csv'      , index_col=0, chunksize=1000, skiprows=2001, dtype=np.float32, header=None)
data_cat_chunks = pd.read_csv('train_categorical_c.csv', index_col=0, chunksize=1000, skiprows=2001, dtype=np.float32, header=None)

X_pos = []
y_pos = []
i = -1
for rows_n in data_num_chunks:
    i+=1
    rows_n.fillna(0., inplace=True)
    
#     rows_c = data_cat_chunks.__next__()
#     rows_c = rows_c.div(cat_features, axis='columns')
    
#     rows = pd.concat([rows_c, rows_n], axis=1)
    rows = rows_n
    for row in rows.values:
        if row[-1] == 0:
            continue
        X_pos.append(row[:-1])
        y_pos.append(row[-1])
    if i %100 == 0:
        print(i)

X_pos = np.array(X_pos).astype(np.float32)
y_pos = np.array(y_pos).astype(int)


0
100
200
300
400
500
600
700
800
900
1000
1100


In [7]:
(X_pos.shape, y_pos.shape)

((6872, 968), (6872,))

In [8]:
with open("valid_pos.pickle", 'wb') as f:
    data = {
        "X_valid":X_valid,
        "y_valid":y_valid,
        "X_pos": X_pos,
        "y_pos":y_pos 
    }
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

In [9]:
data = pickle.load(open("valid_pos.pickle", 'rb'))
X_valid = data["X_valid"]
y_valid = data["y_valid"]
X_pos = data["X_pos"]
y_pos = data["y_pos"]

In [10]:
(X_valid.shape, y_valid.shape, X_pos.shape, y_pos.shape)

((2000, 968), (2000,), (6872, 968), (6872,))

In [11]:

def wb(wshape=[None], bshape=[None], device='/cpu:0'):
    with tf.device(device):
        w = tf.get_variable("w", wshape, initializer=tf.truncated_normal_initializer(stddev=0.1))
        b = tf.get_variable('b', bshape, initializer=tf.constant_initializer(0.0))
    return w, b

In [12]:

def to_one_hot(y, nlabels=None):
    if nlabels == None:
        nlabels = np.unique(y).__len__()
    return (np.arange(nlabels) == y[:,None]).astype(float)

In [13]:

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [14]:
y_valid_oh = to_one_hot(y_valid, nlabels=2)
y_pos_oh = to_one_hot(y_pos, nlabels=2)

(y_valid_oh.shape, y_pos_oh.shape)


((2000, 2), (6872, 2))

In [ ]:
#idx_pos = np.where(y == 1)[0]
# X_test = data_num.iloc[:,1:-1].values[idx_pos]
# y_test = y_oh[idx_pos]

# idx_neg = np.where(y == 0)[0]
# mask = np.random.choice([False, True], len(idx_neg), p=[0.999, 0.001])
# idx_neg = idx_neg[mask]
# print(idx_neg.shape)

# X_neg = data_num.iloc[:,1:-1].values[idx_neg]
# y_neg = y_oh[(idx_neg,)]
# X_test = np.concatenate((X_test, X_neg), axis=0)
# y_test = np.concatenate((y_test, y_neg), axis=0)
# print(X_test.shape, y_test.shape)

In [27]:
batch_size = 16
l2_reg_norm = 5e-5
features = X_valid.shape[1]
train_size = train_len

widest = 1024

tf.reset_default_graph()
graph = tf.Graph()
with graph.as_default():
    X_tf = tf.placeholder(tf.float32, shape=(batch_size, features))
    y_tf = tf.placeholder(tf.float32, shape=(batch_size,2))
    X_valid_tf = tf.constant(X_valid)
    X_pos_tf = tf.constant(X_pos)

    with tf.variable_scope("Layer1"):
            layer1_weights, layer1_biases = wb([features, widest], [widest])
    with tf.variable_scope("Layer2"):
            layer2_weights, layer2_biases = wb([widest, widest//2], [widest//2])
    with tf.variable_scope("Layer3"):
            layer3_weights, layer3_biases = wb([widest//2, widest//2], [widest//2])
#     with tf.variable_scope("Layer4"):
#             layer4_weights, layer4_biases = wb([widest//2, widest//4], [widest//4])
#     with tf.variable_scope("Layer5"):
#             layer5_weights, layer5_biases = wb([widest//4,64], [64])
    with tf.variable_scope("Layer6"):
            layer6_weights, layer6_biases = wb([widest//2, 2], [2])

    def model(data, train=True):
        print("data", data.get_shape())
        
        layer1 = tf.nn.relu(tf.matmul(data, layer1_weights) + layer1_biases)
        print("layer1", layer1.get_shape())
 
            
        layer2 = tf.nn.relu(tf.matmul(layer1, layer2_weights) + layer2_biases)
        print("layer2", layer2.get_shape())
#         if train:
#             tf.nn.dropout(layer2, 0.5)
            
        layer3 = tf.nn.relu(tf.matmul(layer2, layer3_weights) + layer3_biases)
        print("layer3", layer3.get_shape())
        if train:
            tf.nn.dropout(layer3, 0.5)
            
#         layer4 = tf.nn.relu(tf.matmul(layer3, layer4_weights) + layer4_biases)
#         print("layer4", layer4.get_shape())
#         if train:
#             tf.nn.dropout(layer4, 0.5)
            
#         layer5 = tf.nn.relu(tf.matmul(layer4, layer5_weights) + layer5_biases)
#         print("layer5", layer5.get_shape())
        
        
        layer6 = tf.nn.relu(tf.matmul(layer3, layer6_weights) + layer6_biases)
        print("layer6", layer6.get_shape())

        return layer6

    logits = model(X_tf)

    loss_data = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, y_tf))
    regularizers = (tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer1_biases) +
                    tf.nn.l2_loss(layer2_weights) + tf.nn.l2_loss(layer2_biases) +
                    tf.nn.l2_loss(layer3_weights) + tf.nn.l2_loss(layer3_biases) +
#                     tf.nn.l2_loss(layer4_weights) + tf.nn.l2_loss(layer4_biases) + 
#                     tf.nn.l2_loss(layer5_weights) + tf.nn.l2_loss(layer5_biases) + 
                    tf.nn.l2_loss(layer6_weights) + tf.nn.l2_loss(layer6_biases))
    loss_l2 = l2_reg_norm * regularizers
    loss = loss_data + loss_l2

    # Optimizer.
    global_step = tf.Variable(0, trainable=False)
    learn_rate  = tf.train.exponential_decay(.0001, global_step*batch_size, train_size//4, 0.5, staircase=True)
#     tf.scalar_summary('learning_rate', learn_rate)
    optimizer = tf.train.AdamOptimizer(learn_rate).minimize(loss, global_step=global_step, name="Optimizer")

    train_prediction = tf.nn.softmax(logits)
    test_prediction = tf.nn.softmax(model(X_valid_tf, train=False))
    pos_prediction = tf.nn.softmax(model(X_pos_tf, train=False))


data (16, 968)
layer1 (16, 1024)
layer2 (16, 512)
layer3 (16, 512)
layer6 (16, 2)
data (2000, 968)
layer1 (2000, 1024)
layer2 (2000, 512)
layer3 (2000, 512)
layer6 (2000, 2)
data (6872, 968)
layer1 (6872, 1024)
layer2 (6872, 512)
layer3 (6872, 512)
layer6 (6872, 2)


In [28]:


pos_size = 4
batch_size_part = batch_size-pos_size
num_steps = train_len//batch_size_part

data_num_chunks = pd.read_csv('train_numeric.csv'      , index_col=0, chunksize=batch_size_part, skiprows=2001, dtype=np.float32, header=None)
data_cat_chunks = pd.read_csv('train_categorical_c.csv', index_col=0, chunksize=batch_size_part, skiprows=2001, dtype=np.float32, header=None)


print(num_steps)
with tf.Session(graph=graph) as sess:
    init_op = tf.initialize_all_variables()
    saver = tf.train.Saver()
    init_op.run()
    print("Initialized valiables")
    for i in range(num_steps):
        df_n = data_num_chunks.__next__()
        df_n.fillna(0, inplace=True)
#         df_c = data_cat_chunks.__next__()
#         df_c = df_c.div(cat_features, axis='columns')
#         df = pd.concat([df_c, df_n], axis=1)
        df = df_n
        X_ = df.values[:, :-1]
        y_ = to_one_hot(df.values[:, -1], nlabels=2)

        
#         offset = (i * batch_size_part) % (y_train.shape[0] - batch_size_part)
#         y_ = y_train[offset:(offset + batch_size_half)]

        offset_pos = (i*pos_size) % (len(X_pos) - pos_size)
#         add_idx = idx_pos[offset_pos:offset_pos+1]
        y__ = y_pos_oh[offset_pos:offset_pos+pos_size]
        X__ = X_pos[offset_pos:offset_pos+pos_size, :]

        X_ = np.concatenate((X_, X__), axis=0)
        y_ = np.concatenate((y_, y__), axis=0)

        
        p = np.random.permutation(batch_size)
        X_ = X_[p]
        y_ = y_[p]
        
        feed_dict = {X_tf : X_, y_tf : y_}
        _, l, pred = sess.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        if (i < 100 and i%10 == 0) or i%1000 == 0:
            print(i, l, accuracy(pred, y_ ))
        if i>0 and i%10000 == 0:
            print(i, "test", accuracy(test_prediction.eval(), y_valid_oh))
            print(i, "pos", accuracy(pos_prediction.eval(), y_pos_oh))
            print(np.c_[pred, y_])
        if i>0 and i%30000 == 0:
            save_path = saver.save(sess, "bosch_{}.ckpt".format(i))
    save_path = saver.save(sess, "bosch_end.ckpt")
    

98478
Initialized valiables
0 1.02986 68.75
10 0.939223 75.0
20 0.999563 75.0
30 0.975819 75.0
40 1.03992 62.5
50 0.925078 75.0
60 0.780512 75.0
70 0.877441 75.0
80 0.875921 75.0
90 0.819999 75.0
1000 0.708791 75.0
2000 0.856888 68.75
3000 0.676673 75.0
4000 0.727681 75.0
5000 0.686212 75.0
6000 0.763242 75.0
7000 0.705295 75.0
8000 0.817997 75.0
9000 0.632697 75.0
10000 0.711594 75.0
10000 test 99.6
10000 pos 0.0145518044237
[[ 0.8857432   0.11425681  1.          0.        ]
 [ 0.82407445  0.1759256   1.          0.        ]
 [ 0.82715619  0.17284377  0.          1.        ]
 [ 0.5         0.5         0.          1.        ]
 [ 0.5         0.5         1.          0.        ]
 [ 0.5         0.5         0.          1.        ]
 [ 0.84192711  0.15807295  1.          0.        ]
 [ 0.82407445  0.1759256   1.          0.        ]
 [ 0.87333012  0.12666991  1.          0.        ]
 [ 0.92274213  0.07725783  1.          0.        ]
 [ 0.92637938  0.0736206   1.          0.        ]
 [ 0.7874

KeyboardInterrupt: 

In [ ]:
# data_cat = pd.read_csv("train_categorical.csv")
#data_cat.head()

In [20]:
# data_num_test = pd.read_csv("test_numeric.csv")
# data_num_test.fillna(0., inplace=True)
# data_num_test.head()
test_ids = np.genfromtxt ('test_numeric.csv', delimiter=",", skip_header=1, usecols = (0,), dtype=int)

In [21]:
test_ids.shape

(1183748,)

In [ ]:
# X_test_real = data_num_test.iloc[:,1:].values.astype(np.float32)


In [22]:
data_real = pd.read_csv('test_numeric.csv', chunksize=50000, skiprows=1, dtype=np.float32, header=None)

ans = np.array([])
with tf.Session(graph = graph) as sess:
    init_op = tf.initialize_all_variables()
    saver = tf.train.Saver()
    init_op.run()
    # Restore variables from disk.
    saver.restore(sess, "bosch_end.ckpt")
    
    X_tf_test = tf.placeholder(tf.float32, shape=(None, features))
    test_prediction = tf.nn.softmax(model(X_tf_test, train=False))
    
    i = -1
    for data in data_real:
        i+=1
        data.fillna(0., inplace=True)
        ans_ = sess.run(test_prediction, feed_dict = {X_tf_test:data.values[:, 1:]})
        if i == 0:
            ans = ans_
        else:
            ans = np.concatenate((ans, ans_), axis=0)
        print("Step", i, data.shape, len(ans))

        
      

data (?, 968)
layer1 (?, 1024)
layer2 (?, 512)
layer3 (?, 256)
layer6 (?, 2)
Step 0 (50000, 969) 50000
Step 1 (50000, 969) 100000
Step 2 (50000, 969) 150000
Step 3 (50000, 969) 200000
Step 4 (50000, 969) 250000
Step 5 (50000, 969) 300000
Step 6 (50000, 969) 350000
Step 7 (50000, 969) 400000
Step 8 (50000, 969) 450000
Step 9 (50000, 969) 500000
Step 10 (50000, 969) 550000
Step 11 (50000, 969) 600000
Step 12 (50000, 969) 650000
Step 13 (50000, 969) 700000
Step 14 (50000, 969) 750000
Step 15 (50000, 969) 800000
Step 16 (50000, 969) 850000
Step 17 (50000, 969) 900000
Step 18 (50000, 969) 950000
Step 19 (50000, 969) 1000000
Step 20 (50000, 969) 1050000
Step 21 (50000, 969) 1100000
Step 22 (50000, 969) 1150000
Step 23 (33748, 969) 1183748


In [23]:
(ans.shape, test_ids.shape)


((1183748, 2), (1183748,))

In [24]:
ans = np.argmax(ans, axis=1)

In [25]:
result = np.c_[test_ids, ans]

In [ ]:
import csv
with open('answer.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(["Id","Response"])
    for i, a in enumerate(ans1):
        writer.writerow([test_ids[i],a])

In [26]:
np.savetxt('answer.csv',
           result,
           delimiter=',',
           fmt=('%d', '%d'),
           header='Id,Response',
           comments='',
           )